<a href="https://colab.research.google.com/github/ryanraba/casa6/blob/master/CARTA_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download and install CARTA and a test image

In [0]:
import os

CARTA = "CARTA-v1.1-remote"    # keep this updated to desired version

print("downloading an image from CASAguide First Look at Imaging...")
os.system("wget -r -np -nH https://bulk.cv.nrao.edu/synth/dred_workshops/sis14/working_data/sis14_twhya_cont.image/")
os.system("mv synth/dred_workshops/sis14/working_data/sis14_twhya_cont.image /content/.")

print("downloading ngrok...")
os.system("wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip")
os.system("unzip -o ngrok-stable-linux-amd64.zip");

print("downloading " + CARTA + "...")
os.system("wget https://github.com/CARTAvis/carta-releases/releases/download/v1.1/CARTA-v1.1-remote.tar.gz")
os.system("tar -xvf "+CARTA+".tar.gz")
os.system("rm "+CARTA+"/carta-backend/lib/libstdc++.so.6")

print("installing some network libraries...")
os.system("apt-get install iproute2")
os.system("apt-get install iputils-ping")
os.system("apt-get install net-tools")
os.system("rm -r sample_data")
print('complete')

# Execute CARTA backend
This will spawn three processes, one for the CARTA backend and two for https tunneling through the firewall. This allows for remote connection directly to the Jupyter virtual environment runtime.  


Click on the output link to open a new browser tab with the CARTA web app.

In [0]:
import subprocess as sp
import socket
import signal
import time
import requests

FRONT_PORT = '39118'  # make something up
BACK_PORT = '41567'
HOST_IP = socket.gethostbyname(socket.gethostname()) 

cp = sp.Popen(CARTA+'/carta --port='+BACK_PORT + ' --fport='+FRONT_PORT, shell=True, preexec_fn=os.setsid, stdin=sp.PIPE, stdout=sp.PIPE, stderr=sp.STDOUT)
gp1 = sp.Popen('./ngrok http ' + HOST_IP+':'+FRONT_PORT, shell=True, preexec_fn=os.setsid, stdin=sp.PIPE, stdout=sp.PIPE, stderr=sp.STDOUT)
gp2 = sp.Popen('./ngrok http ' + HOST_IP+':'+BACK_PORT, shell=True, preexec_fn=os.setsid, stdin=sp.PIPE, stdout=sp.PIPE, stderr=sp.STDOUT)

time.sleep(2) # give the processes time to start
fcon = requests.get('http://localhost:4040/api/tunnels').json()['tunnels']
bcon = requests.get('http://localhost:4041/api/tunnels').json()['tunnels']
furl = [fcon[ii]['public_url'] for ii in range(len(fcon)) if fcon[ii]['proto'] == 'https'][0]
burl = [bcon[ii]['public_url'] for ii in range(len(bcon)) if bcon[ii]['proto'] == 'https'][0]
print('click this -> ' + furl +'/?socketUrl=' + burl.replace('https','wss'))

In [0]:
# turn it off
os.killpg(os.getpgid(gp1.pid), signal.SIGTERM)
os.killpg(os.getpgid(gp2.pid), signal.SIGTERM)
os.killpg(os.getpgid(cp.pid), signal.SIGTERM)
print("stopped CARTA and the secure tunnels")

In [0]:
# if something goes wrong, you may wish to inspect one of the following:

#cp.stdout.readline().decode('utf-8')
#!netstat -na | grep $FRONT_PORT
#!netstat -na | grep $BACK_PORT